In [1]:
# Auto-summarize text using a rule based model
# Scrape websites for text data using BeautifulSoup
# Use NLTK for munging text-tpkenazation, stopword removal, etc

# Abstract Extraction
# Find the most important words
# Compute a significance score for senences based on words they contain
# Pick the top most significant sentences

In [2]:
# Word Importance
# Authors tend to repeat the words that are important tothe theme of the text
# Word Importance = Word Frequency

In [3]:
# Sentence Significance
# Sentences which encapsulate more of the important words are more significant
# Significance Score = Sum(Word Importance)

In [5]:
# Program Steps:
# 1. Retrieve Text, Download and parse the text from a webpage
# 2. Preprocess Text, Tokenize text inteo sentences and words and remove stopwords
# 3. Extract Sentences, Rank words and sentences

In [ ]:
# Downloading an Article

In [2]:
import requests
from bs4 import BeautifulSoup as BS

In [3]:
articleURL = "https://www.washingtonpost.com/nation/2020/05/18/trump-rick-bright-coronavirus-60-minutes/"

In [25]:
page = requests.get(articleURL)
soup = BS(page.text, "html.parser")
# soup

In [26]:
# soup.find('article')

In [27]:
# soup.find('article').text

In [28]:
text = ' '.join(map(lambda p: p.text, soup.find_all('article')))
# text

In [32]:
text.encode('ascii', errors = 'replace').replace(b"?",b" ")

b"President Trump walks across the South Lawn of the White House on Sunday after returning to Washington from the Camp David presidential retreat. (Mandel Ngan/AFP/Getty Images)By Timothy Bella closeTimothy BellaDeputy editor for Morning MixEmailEmailBioBioFollowFollowMay 18, 2020 at 6:04 AM EDTAfter former top vaccine official Rick Bright maintained Sunday that the federal government s response to the coronavirus pandemic has been slow and chaotic, President Trump responded in a late-night tweetstorm, saying that whistleblowers like Bright are  causing great injustice and harm  to the nation.In an interview with  60 Minutes,  Bright, the former director of the Biomedical Advanced Research and Development Authority, criticized the government for failing to have a clear plan in place to address the totality of the American outbreak. He reiterated claims that he was reassigned to a less respected role within the Department of Health and Human Services last month after raising concerns re

In [19]:
def getTextWaPo(url):
    page = requests.get(url)
    soup = BS(page.text, "html.parser")
    text = ' '.join(map(lambda p: p.text, soup.find_all('article')))
    return text.encode('ascii', errors = 'replace').replace(b"?",b" ")    

In [20]:
text = getTextWaPo(articleURL)

In [ ]:
# Preprocessing Article Text

In [33]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation

In [46]:
sents = sent_tokenize(text)
# sents

In [43]:
word_sent = word_tokenize(text.lower())
# word_sent

In [50]:
_stopwords = set(stopwords.words('english') + list(punctuation) + ['“', '”', '’'] )
# _stopwords

In [51]:
word_sent = [word for word in word_sent if word not in _stopwords]
# word_sent

In [39]:
# Extract a summary

In [52]:
from nltk.probability import FreqDist

In [53]:
freq = FreqDist(word_sent)
freq

FreqDist({'bright': 14, 'coronavirus': 12, 'trump': 11, 'government': 7, 'drug': 7, 'whistleblower': 7, 'hydroxychloroquine': 6, 'pandemic': 5, '60': 5, 'health': 5, ...})

In [54]:
from heapq import nlargest

In [56]:
nlargest(10, freq, key = freq.get)

['bright',
 'coronavirus',
 'trump',
 'government',
 'drug',
 'whistleblower',
 'hydroxychloroquine',
 'pandemic',
 '60',
 'health']

In [58]:
from collections import defaultdict

In [61]:
# Keys -> Sentences
# Values -> Significance Scores
ranking = defaultdict(int)

for i, sent in enumerate(sents):
    for w in word_tokenize(sent.lower()):
        if w in freq:
            ranking[i] += freq[w]
ranking

defaultdict(int,
            {0: 35,
             1: 187,
             2: 138,
             3: 28,
             4: 90,
             5: 106,
             6: 32,
             7: 51,
             8: 18,
             9: 123,
             10: 73,
             11: 243,
             12: 28,
             13: 16,
             14: 259,
             15: 17,
             16: 55,
             17: 12,
             18: 187,
             19: 20})

In [63]:
sents_idx = nlargest(4, ranking, key = ranking.get)
sents_idx

[14, 11, 1, 18]

In [64]:
[sents[j] for j in sorted(sents_idx)]

['(Mandel Ngan/AFP/Getty Images)By Timothy Bella\xa0closeTimothy BellaDeputy editor for Morning MixEmailEmailBioBioFollowFollowMay 18, 2020 at 6:04 AM EDTAfter former top vaccine official Rick Bright maintained Sunday that the federal government’s response to the coronavirus pandemic has been slow and chaotic, President Trump responded in a late-night tweetstorm, saying that whistleblowers like Bright are “causing great injustice and harm” to the nation.In an interview with “60 Minutes,” Bright, the former director of the Biomedical Advanced Research and Development Authority, criticized the government for failing to have a clear plan in place to address the totality of the American outbreak.',
 "It could have negative side effects,” he told “60 Minutes.” “And it could even lead to death.”As Bright outlined in his 89-page whistleblower complaint, he received a directive from the White House to “drop everything” to make the drug “widely available to the American public.” He described th

In [65]:
# more goes here